In [1]:
from meta_planning.parsers import parse_trajectory, parse_model, parse_plan, parse_problem
from meta_planning import ValidationTask
from meta_planning import dataset
from meta_planning.pddl import SensorModel, Plan, Literal
from meta_planning.observations import State, Observation
from meta_planning.functions import generate_all_literals, get_matching_literals
from meta_planning.generator import generate_trajectory

import numpy as np
import glob
from random import random
from collections import defaultdict
import time

In [2]:
domain = "grid"
agents = 1

M = parse_model("benchmarks/%s/domain-%s.pddl" % (domain, agents))

In [3]:
# Sensor Model

d = defaultdict(dict)

max_x = 4
max_y = 4

for a in range(agents):

    at_literals = [Literal("at-r%s" % (a+1), ["loc-x%s-y%s" % (i,j)], True) for i in range(max_x+1) for j in range(max_y+1)]

    for i in range(max_x+1):
        for j in range(max_y+1):
            for lit in at_literals:
                d[Literal("at-r%s" % (a+1), ["loc-x%s-y%s" % (i,j)], True)][lit] = 0


    for i in range(max_x+1):
        for j in range(max_y+1):

            probs = [0.8,0.05,0.05,0.05,0.05]

            if i == 0:
                probs[1] = 0
            if i == max_x:
                probs[2] = 0
            if j == 0:
                probs[3] = 0
            if j == max_y:
                probs[4] = 0

            sum_probs = sum(probs)
            probs = [p/sum_probs for p in probs]

            d[Literal("at-r%s" % (a+1), ["loc-x%s-y%s" % (i,j)], True)][ Literal("at-r%s" % (a+1), ["loc-x%s-y%s" % (i,j)], True)] = probs[0]

            if probs[1]!=0:
                d[Literal("at-r%s" % (a+1), ["loc-x%s-y%s" % (i,j)], True)][ Literal("at-r%s" % (a+1), ["loc-x%s-y%s" % (i-1,j)], True)] = probs[1]
            if probs[2]!=0:
                d[Literal("at-r%s" % (a+1), ["loc-x%s-y%s" % (i,j)], True)][ Literal("at-r%s" % (a+1), ["loc-x%s-y%s" % (i+1,j)], True)] = probs[2]
            if probs[3]!=0:
                d[Literal("at-r%s" % (a+1), ["loc-x%s-y%s" % (i,j)], True)][ Literal("at-r%s" % (a+1), ["loc-x%s-y%s" % (i,j-1)], True)] = probs[3]
            if probs[4]!=0:
                d[Literal("at-r%s" % (a+1), ["loc-x%s-y%s" % (i,j)], True)][ Literal("at-r%s" % (a+1), ["loc-x%s-y%s" % (i,j+1)], True)] = probs[4]



S = SensorModel(d, probabilistic=True)

In [4]:
problems = sorted(glob.glob("benchmarks/%s/PGRTI/%s/*" % (domain, agents)))
num_hyps = 5

for problem in problems:
    t = parse_trajectory(problem + "/trajectory" , M)
    
    for observability in [0.3, 0.5, 0.7]:
           
        costs = []
        times = []

        # Generate observation
        o = t.observe_with_sensor_model(S, action_observability=0)
        
        states = [o.states[0]] + [s for s in o.states[1:-1] if random() < observability] + [o.states[-1]]
        o = Observation(o.objects, states, False, False)

        for i in range(num_hyps):

            H = parse_problem(problem + "/hyp-%s.pddl" % i, M)

            o.states[0] = H.init

            O = [o]

            tic = time.time()
            # Solve with sensor model
            valtask = ValidationTask(M,O, compile_known=False, sensor_model=S, use_cost=True)
            sol = valtask.validate(clean=True, planner="metric-ff", t=600)
            
            toc = time.time()
            times.append(toc-tic)

            try:
                trajectory = generate_trajectory(valtask.compiled_model, valtask.compiled_problem, sol.solution_plan)
                costs.append(trajectory.cost)
            except:
                costs.append(9999)


        f = open(problem + "/results-Prob-%.1f" % observability, "w")
        f.write(" ".join(map(str, costs)))
        f.close()
        
        f = open(problem + "/times-Prob-%.1f" % observability, "w")
        f.write(" ".join(map(str, times)))
        f.close()


ulimit -t 600; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 600; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 600; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 600; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 600; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 600; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 

ulimit -t 600; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 600; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 600; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 600; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 600; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 600; /home/dieaigar/PhD/meta-planning/src/meta_planning/util/planners/metric-FF/ff -E -O -g 1 -h 0 -o compiled_domain -f compiled_problem -s solution_plan
ulimit -t 